In [ ]:
import nibabel as nib
import numpy as np
import json
from nilearn.image import resample_to_img


fmri_img = nib.load("SDSU_0050182.nii.gz")
atlas_img = nib.load("BN_Atlas_246_1mm.nii.gz")

fmri_data = fmri_img.get_fdata() 
atlas_resampled = resample_to_img(atlas_img, fmri_img, interpolation="nearest")
atlas_data = atlas_resampled.get_fdata().astype(int)

region_ids = np.unique(atlas_data)
region_ids = region_ids[region_ids != 0]

timeseries = {}

for rid in region_ids:
    mask = (atlas_data == rid)
    region_voxels = fmri_data[mask, :] # (voxels, time)
    
    if region_voxels.size == 0: continue

    # Calculate Mean and Z-Score
    mean_ts = region_voxels.mean(axis=0)
    # Z-score: (value - mean) / std. Now 0 is 'average' activity.
    z_ts = (mean_ts - np.mean(mean_ts)) / (np.std(mean_ts) + 1e-8)
    
    timeseries[str(int(rid))] = z_ts.tolist()

with open("timeseries.json", "w") as f:
    json.dump(timeseries, f)



Success! Extracted 246 Z-scored regions.
